## Derive plotwise metrics with pyForMetrics

In this tutorial, we will derive some LiDAR metrics for forest inventory (FI) plots.

First, we ensure that the required packages are installed:

In [1]:
!python -m pip install pyForMetrix
!python -m pip install geopandas wget

We then need to gather some data. In this example, we use data from Germany (Open Geodata Thüringen, ©GDI-Th, [dl-de/by-2-0](http://www.govdata.de/dl-de/by-2-0)). They can be downloaded from the [geodata portal of the State of Thuringia](https://www.geoportal-th.de/de-de/Downloadbereiche/Download-Offene-Geodaten-Th%C3%BCringen) (in German). **Approximate download size: 120 MB**

In [8]:
import os, wget, zipfile
if not os.path.exists('las_623_5718_1_th_2014-2019.laz'):
    if not os.path.exists('data_netzkater.zip'):
        print('Downloading file')
        wget.download('https://geoportal.geoportal-th.de/hoehendaten/LAS/las_2014-2019/las_623_5718_1_th_2014-2019.zip', 'data_netzkater.zip')
    print('Unzipping file')
    zipfile.ZipFile('data_netzkater.zip').extractall('.')
print('Ready!')

Ready!


We then use `laspy` to load the file. As the input point cloud is not normalized by height, we first use a utility function in `pyForMetrics.normalizer` to do that for us.


In [9]:
import laspy
file = laspy.read(r"las_623_5718_1_th_2014-2019.laz")
points = {
    'points': file.xyz,
    'classification': file.classification,
    'scan_angle_rank': file.scan_angle_rank
}

from pyForMetrix.normalizer import normalize
points = normalize(points, distance=5)

Rasterizing for point normalization...


Normalizing LiDAR points: 100%|██████████| 40000/40000 [00:03<00:00, 11934.24it/s]


Now we load some polygons from a shapefile. These polygons represent circular areas, for which e.g. forest inventories have been carried out, but any valid polygon shape may be used. Our data is stored in a [GeoPackage](https://www.geopackage.org/) file, but any file supported by [GeoPandas](https://geopandas.org/en/stable/) or [Shapely](https://shapely.readthedocs.io/en/stable/manual.html) will work.

In [1]:
import geopandas as gpd
plots = gpd.GeoDataFrame.from_file(r"netzkater_polygons.gpkg")
print(plots)

                                            geometry
0  POLYGON ((623210.094 5718429.347, 623210.781 5...
1  POLYGON ((623257.611 5718187.129, 623258.297 5...
2  POLYGON ((623700.325 5718631.002, 623701.012 5...
3  POLYGON ((623709.596 5718801.366, 623710.283 5...
4  POLYGON ((623910.093 5718108.321, 623910.779 5...
5  POLYGON ((623234.432 5718588.122, 623235.119 5...
6  POLYGON ((623516.054 5718391.102, 623516.740 5...


Now we will calculate metrics for each of these plots. In this example, we use a combination of different metrics in the `types` namespace. As we don't have an index file (.lax) for this input file, scanning though all the points may take a minute or two. If you have [LAStools](https://rapidlasso.com/lastools/) installed, you can create an index file by running `lasindex -i las_623_5718_1_th_2014-2019.laz`.

In [2]:
from pyForMetrix.metrix import PlotMetrics
from pyForMetrix.metricCalculators.types import *

pm = PlotMetrics(["las_623_5718_1_th_2014-2019.laz"], plots)
mc = [MCalc_DensityMetrics(), MCalc_HeightMetrics(), MCalc_VarianceMetrics()]
metr = pm.calc_custom_metrics(mc)
print(metr)

Scanning input files to find polygon plots:   0%|                                                                               | 0/1 [00:00<?, ?it/s]

File las_623_5718_1_th_2014-2019.laz does not have a corresponding .lax index file. Expect much slower performance.
Run `lasindex -i las_623_5718_1_th_2014-2019.laz` to create an index file (requires LAStools installation)


Calculating metrics: 100%|██████████| 7/7 [00:00<00:00, 386.57it/s]

   d10  d20  d30  d40  d50  d60  d70  d80  d90  d100  ...       p70       p80  \
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   0.0  ...  390.7311  392.3204   
1  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   0.0  ...  468.9515  470.5180   
2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   0.0  ...  406.6710  407.7968   
3  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   0.0  ...  434.1044  435.8306   
4  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   0.0  ...  377.1825  379.0810   
5  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   0.0  ...  325.9076  326.1004   
6  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   0.0  ...  327.1171  327.3150   

        p90     p100      h_mean   h_stddev   h_absdev    h_skew  h_kurtosis  \
0  393.7132  400.546  382.338601   9.024213   8.644587  0.153006   -1.736357   
1  471.9530  474.233  458.965645  10.989637  10.410608 -0.085498   -1.753655   
2  408.8468  413.032  397.055385  10.786964  10.365036 -0.177707   -1.769685   
3  438.1616  441.869  421.79496